<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option("display.precision", 2)

In [2]:
RAND = 42

In [3]:
df = pd.read_csv('/datasets/insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков нет, возраст и зарплата хранятся в `float`, но разумнее хранить в `int`.  

Переименуем колонки и преобразуем типы:

In [5]:
df.columns = ['sex', 'age', 'income', 'family_members', 'insurance_payments']
df[['age', 'income']] = df[['age', 'income']].astype('int64')
df.head()

,sex,age,income,family_members,insurance_payments
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


In [6]:
df.describe()

,sex,age,income,family_members,insurance_payments
count,5000.0,5000.00,5000.00,5000.00,5000.00
mean,0.5,30.95,39916.36,1.19,0.15
std,0.5,8.44,9900.08,1.09,0.46
min,0.0,18.00,5300.00,0.00,0.00
25%,0.0,24.00,33300.00,0.00,0.00
50%,0.0,30.00,40200.00,1.00,0.00
75%,1.0,37.00,46600.00,2.00,0.00
max,1.0,65.00,79000.00,6.00,5.00


### Вывод:

Зарплата объектов варьируется от 5300 до 79000, при этом большей части объектов страховые выплаты сделаны не были, тогда как некоторым сделаны по 5 раз. Видно, что в семьях в основном 1-2 человека (преобладают семьи без детей), средний возраст в выборке - 30 лет.   
Данные приведены к нужному виду, изучены и готовы к работе.

## Умножение матриц

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?  
a. Изменится. Приведите примеры матриц.  
b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** качество линейной регрессии не изменится

**Обоснование:** 
$$ 
w^{*} = ((XP)^T (XP))^{-1} (XP)^T y = (P^T X^T X P)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$ 
w^{*} = P^{-1} (X^T X)^{-1} X^T y = P^{-1} w
$$

$$
a^{*} = X^{*} w^{*} = X P P^{-1} w = X w = a
$$
Что и требовалось доказать.

<div class="alert alert-block alert-success">
✔️ Вопрос со звёздочкой: а что будет, если домножить $X$ на $X^{-1}$? Естественно, в допущении, что $X$ квадратная и обратная к ней определена. Останутся ли веса $w$ такими же или изменятся?
</div>

При обратимости $X$:
$$
w = X^{-1} y
$$
При домножении на $X^{-1}$:
$$
w^{*} = y
$$
При этом:
$$
a^{*} = X^{*} w^{*} = X X^{-1} X w = X w = a
$$

## Алгоритм преобразования

**Алгоритм**

Возьмем алгоритм домножения признаков на обратимую матрицу. Выяснили, что качество линейной регрессии при этом не меняется.  
Если знать обратную матрицу P (матрица на которую мы умножали признаки), то можно восстановить изначальные признаки при необходимости.

**Обоснование**

См. доказательство в пункте 2.

## Проверка алгоритма

Напишем класс для линейной регрессии с дефолтным признаком, а также с домноженным на обратимую матрицу:

In [7]:
class LinearReg:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T @ X) @ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [16]:
class LinearRegEncoded:
    def fit(self, train_features, train_target):
        try:
            rnd = np.random.RandomState(RAND)
            X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
            P = rnd.normal(size=(X.shape[1], X.shape[1]))
            invP = np.linalg.inv(P)
            self.P = P
            y = train_target
            w = np.linalg.inv(P) @ np.linalg.inv(X.T @ X) @ X.T @ y
            self.w = w
        except:
            print('Домноженная матрица необратима')
    def predict(self, test_features):
        X_test = np.concatenate((np.ones((test_features.shape[0], 1)), test_features), axis=1)
        return X_test.dot(self.P).dot(self.w) 

Обучим модели и посмотрим качество:

In [17]:
features = df.drop('insurance_payments', axis=1)
target = df['insurance_payments']

In [18]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=RAND
)

Наша модель:

In [19]:
rnd = np.random.RandomState(RAND)

In [20]:
model = LinearReg()
model.fit(features_train, target_train)
predictions = model.predict(features_train)
print('r2_score: ', r2_score(target_train, predictions))

r2_score:  0.42439431373646785


In [21]:
model_encoded = LinearRegEncoded()
model_encoded.fit(features_train, target_train)
predictions = model_encoded.predict(features_train)
print('r2_score: ', r2_score(target_train, predictions))

r2_score:  0.4243943137362709


Модель из sklearn:

In [22]:
classic_model = LinearRegression()
classic_model.fit(features_train, target_train)
predictions = classic_model.predict(features_train)
print('r2_score: ', r2_score(target_train, predictions))

r2_score:  0.42439431373646785


In [23]:
P = np.random.RandomState(RAND).normal(size=(features_train.shape[1], features_train.shape[1]))

classic_model = LinearRegression()
classic_model.fit(
    features_train @ P,
    target_train
)
predictions = classic_model.predict(features_train @ P)
print('r2_score: ', r2_score(target_train, predictions))

r2_score:  0.42439431373647263


### Вывод

Качество модели с обычными признаками и домноженными на обратимую матрицу одинаково, что на нашей модели, что на модели из sklearn.